In [1]:
import pandas as pd
import numpy as np 

import geopandas as gpd
import shapely
import fiona
import folium

In [2]:
# utils
def choropleth(grid, data, color_scale, location, zoom):
    f = folium.Figure(width=970, height=300)
    m = folium.Map(
        location=location,
        tiles='Stamen Terrain',
        zoom_start=zoom
    ).add_to(f)
    data.index = data.index.map(str) # bug prevention
    m.choropleth(geo_data=grid, 
                 data=data,
                 threshold_scale=color_scale,
                 key_on="feature.id",
                 fill_color='Spectral_r',line_weight=2)
    return m

def kGrid(points_df, k):
    from sklearn.cluster import KMeans
    kmeans = KMeans(K, random_state=1).fit(points_df[['lat','lon']])
    pdf_copy = points_df.copy()
    pdf_copy['k'] = kmeans.labels_
    pdf_copy['geometry'] = points_df['geometry'].apply(lambda x: [x])
    kgrid = gpd.GeoDataFrame(pdf_copy.groupby('k').agg({'geometry':'sum'}))
    kgrid['geometry'] = [shapely.geometry.MultiPoint(x).convex_hull for x in kgrid.geometry]
    kgrid = kgrid.loc[[type(x)==shapely.geometry.Polygon for x in kgrid.geometry]] #excluding Points and Linestrings grid
    kgrid = kgrid.loc[kgrid.area>1e-6]
    kgrid = gpd.GeoDataFrame(kgrid, geometry=kgrid['geometry'])
    kgrid.crs = {'init': 'epsg:4326'}
    kgrid = kgrid.to_crs(fiona.crs.from_epsg(4326))
    return kgrid

def rectangular_grid(xcells, ycells, points_df):
    xmin = points_df['lon'].min()
    xmax = points_df['lon'].max()
    ymin = points_df['lat'].min()
    ymax = points_df['lat'].max()

    xwindow = (xmax-xmin)/xcells
    ywindow = (ymax-ymin)/ycells

    x0, y0 = xmin, ymax
    pols = []
    for w in range(ycells):
        for h in range(xcells):
            pols.append(shapely.geometry.Polygon([(x0,y0),(x0+xwindow,y0),(x0+xwindow,y0-ywindow),(x0,y0-ywindow)]))
            x0 += xwindow
        y0 -= ywindow
        x0 = xmin
    rg = gpd.GeoDataFrame(geometry=pols, crs={'init': 'epsg:4326'}).to_crs(fiona.crs.from_epsg(4326))
    rg.index = rg.index.map(str)
    return rg 

def cscale(data):
    scale = []
    for i in [0,0.50,0.80,0.95,1]:
        if i==0:
            if data.quantile(i)==0:
                scale.append(0)
            else:
                scale.append(data.quantile(i)-1)
        else:
            scale.append(data.quantile(i)+1)
    return scale

def create_join_hash(meta, grid):
    # returns dataframe with 'index_grid' and sensor name relationship
    return gpd.sjoin(meta, grid, rsuffix='grid' ,op='intersects').set_index('name')

def grid_medians(var, freq, sensors, join_hash):
    # sensors is the dataframe with the columns ['Variable', 'Timestamp', 'Sensor Name']
    # join hash is the dataframe with 'index_grid' and sensor name relationship
    svalues = sensors.loc[sensors['Variable']==var]
    svalues = svalues.join(join_hash['index_grid'],on='Sensor Name').dropna()

    svalues['Timestamp'] = pd.to_datetime(svalues['Timestamp'])
    svalues = svalues.set_index(['index_grid','Sensor Name','Timestamp'])

    level_values = svalues.index.get_level_values
    result = (svalues.groupby([level_values(i) for i in [0,1]]
                          +[pd.Grouper(freq=freq, level=-1)]).median())
    result = result.groupby(level=[0,2]).median()
    return result.reset_index()

In [60]:
# feature extraction utils
def kneighbors_sensors(point, k, join_hash):
    ix = join_hash['geometry'].apply(lambda x: point.distance(x)).sort_values()[:k].index
    return join_hash.loc[ix].index.values

# def random_sensors():

def sensors_median(sensors, var, freq):
    svalues = sensors.loc[sensors['Variable']==var]
    return svalues.set_index('Timestamp').groupby('Sensor Name').resample(freq).mean()

def propagate_median(grid, sensors, join_hash, var, freq, method, k=5):
    sm = sensors_median(sensors, var, freq)
    if method=='kneighbors':
        res = grid.apply(lambda x: sm.loc[kneighbors_sensors(x['geometry'].centroid, k, join_hash)].groupby('Timestamp').mean(),axis=1)
    elif method=='random':
        res = grid.apply(lambda x: sm.loc[kneighbors_sensors(x['geometry'].centroid, k, join_hash)].groupby('Timestamp').mean(),axis=1)        
    result = pd.DataFrame()
    for i in range(0,len(grid)):
        tmp = res[i].reset_index()
        tmp['index_grid'] = i
        result = pd.concat([result,tmp])
    return result

In [4]:
SHAPE_FOLDER = '/home/adelsondias/Repos/newcastle/air-quality/shape/Middle_Layer_Super_Output_Areas_December_2011_Full_Extent_Boundaries_in_England_and_Wales'
DATA_FOLDER = '/home/adelsondias/Repos/newcastle/air-quality/data_allsensors_8days'

In [5]:
sensors = pd.read_csv(DATA_FOLDER+'/data.csv')
metadata = pd.read_csv(DATA_FOLDER+'/sensors.csv')
metadata.head()

,name,base_height,latest,source,type,active,sensor_height,lon,lat
0,new_new_emote_1172,97.358508,2018-07-16 15:20:35,"{'third_party': False, 'web_display_name': 'Em...",Air Quality,True,None,-1.595367,54.986412
1,new_new_emote_2402,-999.000000,2018-07-16 04:59:39,"{'third_party': False, 'web_display_name': 'Em...",Air Quality,True,-999,-1.616822,54.981297
2,new_new_emote_2605,-999.000000,2018-07-16 15:20:28,"{'third_party': False, 'web_display_name': 'Em...",Air Quality,True,-999,-1.618207,54.975402
3,new_new_emote_901,38.634146,2018-07-16 15:19:46,"{'third_party': False, 'web_display_name': 'Em...",Air Quality,True,3.5,-1.623169,54.968311
4,new_new_emote_1708,46.019822,2018-07-16 15:19:59,"{'third_party': False, 'web_display_name': 'Em...",Air Quality,True,3.2,-1.622541,54.970349


In [6]:
# temporal parsing
sensors['Timestamp'] = pd.to_datetime(sensors['Timestamp'])

# spatial parsing - geometry column
metadata = gpd.GeoDataFrame(metadata[['name','type','active','lon','lat']],
                        geometry=[shapely.geometry.Point(xy) for xy in zip(metadata['lon'], metadata['lat'])], 
                        crs={'init': 'epsg:4326'}).to_crs(fiona.crs.from_epsg(4326))

In [98]:
# # only sensors for Newcastle
lsoa = gpd.read_file(SHAPE_FOLDER+'/Middle_Layer_Super_Output_Areas_December_2011_Full_Extent_Boundaries_in_England_and_Wales.shp')
lsoa = lsoa[lsoa['msoa11nm'].str.contains('Newcastle upon Tyne')]
lsoa = lsoa.to_crs(fiona.crs.from_epsg(4326))
lsoa.crs = {'init': 'epsg:4326', 'no_defs': True}

metadata = gpd.sjoin(metadata, lsoa, how='inner' ,op='intersects')[['name','type','active','lon','lat','geometry']]
nucsensors = metadata['name'].unique()
sensors = sensors.apply(lambda x: x if x['Sensor Name'] in nucsensors else np.nan, axis=1)

In [103]:
# grid creation
xcells, ycells = 20, 20
grid = rectangular_grid(xcells, ycells, metadata)

# spatial join
join = create_join_hash(metadata, grid)

# map with all sensors
data = join['index_grid'].value_counts()
choropleth(grid=grid, 
           data=data, 
           color_scale=cscale(data), location=[55.00 ,-1.50], zoom=11)

# heatmap for the sensor count in Newcastle upon Tyne:

In [ ]:
# only sensors for NO2
no2sensors = sensors.loc[sensors['Variable']=='NO2','Sensor Name'].unique()
no2meta = metadata.apply(lambda x: x if x['name'] in no2sensors else np.nan, axis=1).dropna()
no2sensors = sensors.apply(lambda x: x if x['Sensor Name'] in no2sensors else np.nan, axis=1).dropna()

In [82]:
# join = gpd.sjoin(no2meta, grid,  rsuffix='grid' ,op='intersects').set_index('name')
join = create_join_hash(no2meta, grid)
data = join['index_grid'].value_counts()

choropleth(grid=grid, 
           data=data, 
           color_scale=cscale(data), location=[55.00 ,-1.50], zoom=11)

# heatmap for NO2 sensors counts:

Ok, so we need to predict NO_2 values for these blue cells.

Although we can increase the number of cells that we have data (and increase training data), by controlling the grid size, the consequences for that aren't very clear so far. For sure, the number of sensors in each cell will decreae, but is that so bad? (**open problem to be instestigated, it will stay in the air for now**)

How can we extract features (first, endogenous ones) for each cell?

In [9]:
# idea 1
# variable: median NO2 of all sensors within each grid cell, indexed by timestamp
r = grid_medians('NO2', 'H', no2sensors, join)

timestamp = '2018-07-09 01:00:00'
data = r[r['Timestamp']==timestamp].set_index('index_grid')['Value']

choropleth(grid=grid, 
           data=data, 
           color_scale=cscale(data), location=[55.00 ,-1.50], zoom=11)

In [86]:
# idea 2
# variable: median NO2 of k-neighbors sensors for each grid cell, indexed by timestamp

result = propagate_median(grid, no2sensors, join, 'NO2', 'H', 'kneighbors')

timestamp = '2018-07-09 09:00:00'
data = result[result['Timestamp']==timestamp].set_index('index_grid')['Value']
data.index = data.index.map(str)

choropleth(grid=grid, 
           data=data, 
           color_scale=cscale(data), location=[55.00 ,-1.50], zoom=11)

In [35]:
# # idea 3
# # variable: median NO2 of k random sensors for each grid cell, indexed by timestamp
